# Create a mpi4py environement and run it <br>across multiple nodes
This notebook shows how you can run MPI programs across multiple nodes<br>
from within a notebook.  We provide a script to build  the environment and<br>
a script to launch it.  We also provide a convience script for creating a <br>
tunnel from your desktop machine to a compute node.

We also show that you can compile mpi programs while you are in the notebook.

## Note this environemnt contains the slurm magic commands, shown below.

In [1]:
%load_ext slurm_magic

In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %sacct  %sacctmgr  %salloc  %sattach  %save  %sbatch  %sbcast  %sc  %scancel  %scontrol  %sdiag  %set_env  %sinfo  %slurm  %smap  %sprio  %squeue  %sreport  %srun  %sshare  %sstat  %store  %strigger  %sview  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls

## Here's the script that created this environment.<br>It can be obtained from:
###### conda is needed for git on compute nodes
`
ml conda
git clone https://github.com/NREL/HPC.git ~/HPC
`


In [3]:
cat ~/HPC/slurm/source/jupyter.sh

:<<++++

Author: Tim Kaiser

Build a new version of python mpi4py.
Works with IntelMPI, MPT, and OpenMPI, just change the module load commands.


USAGE:
    source jupyter.sh

 To use the new version after the initial Install
   module load conda
   source activate
   source activate $MYVERSION
   module load gcc/8.4.0 
   #module load intel-mpi/2020.1.217
   module load mpt

++++

### Build a new version of python with and mpt MPI version of mpi4py
CWD=`pwd`
export MYVERSION=dompt
cd ~
module load conda 2> /dev/null || echo "module load conda failed"
conda create --name $MYVERSION python=3.8 jupyter matplotlib scipy pandas xlwt dask -y

### Don't do conda init
### Just do source activate
source activate 
source activate $MYVERSION

which pip
which python

### Install mpi4py
module load gcc/8.4.0  2> /dev/null || echo "module load gcc failed"
#module load intel-mpi/2020.1.217  2> /dev/null || echo "module load mpi failed"
module load mpt  2> /dev/null || echo "module load mpi failed"
p

### The examples shown below, as well as a vesion of the slurm script used to start this notebook can be obtained via
`
git clone https://github.com/NREL/HPC
cd mpi/mpi4py
`

## Here is the script that started this notebook

In [4]:
cat start_jupyter.sh 

#!/bin/bash
#SBATCH --job-name="jupyter"
#SBATCH --nodes=2
#SBATCH --account=hpcapps
#SBATCH --partition=debug
#SBATCH --time=01:00:00
##SBATCH --mail-type=ALL
##SBATCH --mail-user=tkaiser2@nrel.gov

#see ../../../slurm/source/jupyter.sh
#for a script to create the conda 
#environment referenced below
export MYVERSION=dompt
module load conda
source activate
source activate $MYVERSION
module load gcc/8.4.0 
module load mpt

date      > ~/jupyter.log
hostname >> ~/jupyter.log
jupyter notebook --NotebookApp.password='' --no-browser  >> ~/jupyter.log 2>&1




<hr> 

### After the job starts the file ~/jupyter.log will contain the information required to  connect to the notebook.  What we need is the node name, port number, and the html string.  <br><br>The bash function, tunnel, also available from the git repository mentioned above can provide a shortcut for connecting to the notebook.  After it is installed in your bash environment the syntax is 

`
tunnel node_name port_number
`
### After it connects you copy/past the html string from ~/jupyter.log into a browser

<hr> 

### Let's look at what we have



In [5]:
ls

bonnkit.ipynb.save                 optics.ipynb
c_ex02.c                           report.py*
checkpoint                         save/
checkpoints-1.data-00000-of-00001  slurm-6688638.out
checkpoints-1.data-00000-of-00002  slurm-6688667.out
checkpoints-1.data-00001-of-00002  slurm-6688673.out
checkpoints-1.index                slurm-6688677.out
cupy.ipynb                         slurm-6688703.out
dojup2g                            slurm-6694076.out
dojup2x*                           slurm-6695108.out
dompi.ipynb                        slurm-6696387.out
dpcu.ipynb                         slurm-6705602.out
dpcu.ipynb.save                    slurm-6705733.out
getver*                            slurm-6705788.out
junk/                              slurm-6705811.out
libcublas.so.10@                   slurm-6711763.out
libcudart.so.10.1@                 slurm-6711786.out
libcufft.so.10@                    slurm-6712205.out
libcurand.so.10@                   slurm-6712294.out
libcusolver.so.10

In [6]:
squeue -u tkaiser2

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)
0,6713691,debug,jupyter,tkaiser2,R,1:25,2,r102u[34-35]


<hr> 

### Get a list of our hosts we have available

In [7]:
srun -n 4 --tasks-per-node=2 hostname

'r102u34\nr102u35\nr102u34\nr102u35\n'

In [8]:
out=_
out=out.split()
#print(out)
for o in out :
    print(o)

r102u34
r102u35
r102u34
r102u35


In [10]:
cd /home/tkaiser2/newdriver

/home/tkaiser2/newdriver


<hr> 

### The examples shown below, as well as a vesion of the slurm script given above can be obtained via
`
git clone https://github.com/NREL/HPC
cd mpi/mpi4py
`

In [11]:
ls

bonnkit.ipynb.save                 optics.ipynb
c_ex02.c                           report.py*
checkpoint                         save/
checkpoints-1.data-00000-of-00001  slurm-6688638.out
checkpoints-1.data-00000-of-00002  slurm-6688667.out
checkpoints-1.data-00001-of-00002  slurm-6688673.out
checkpoints-1.index                slurm-6688677.out
cupy.ipynb                         slurm-6688703.out
dojup2g                            slurm-6694076.out
dojup2x*                           slurm-6695108.out
dompi.ipynb                        slurm-6696387.out
dpcu.ipynb                         slurm-6705602.out
dpcu.ipynb.save                    slurm-6705733.out
getver*                            slurm-6705788.out
junk/                              slurm-6705811.out
libcublas.so.10@                   slurm-6711763.out
libcudart.so.10.1@                 slurm-6711786.out
libcufft.so.10@                    slurm-6712205.out
libcurand.so.10@                   slurm-6712294.out
libcusolver.so.10

<hr> 

### Run a mpi4py program and put output in a file

In [12]:
srun -n 72 --tasks-per-node=36 -o report.out ./report.py

''

In [13]:
cat report.out

xxxxxx Hello from 29 on r102u34   ,  29   
xxxxxx Hello from 38 on r102u35   ,   2   
xxxxxx Hello from 48 on r102u35   ,  12   
xxxxxx Hello from 37 on r102u35   ,   1   
xxxxxx Hello from 40 on r102u35   ,   4   
xxxxxx Hello from 51 on r102u35   ,  15   
xxxxxx Hello from 52 on r102u35   ,  16   
xxxxxx Hello from 61 on r102u35   ,  25   
xxxxxx Hello from 41 on r102u35   ,   5   
xxxxxx Hello from 42 on r102u35   ,   6   
xxxxxx Hello from 44 on r102u35   ,   8   
xxxxxx Hello from 47 on r102u35   ,  11   
xxxxxx Hello from 49 on r102u35   ,  13   
xxxxxx Hello from 50 on r102u35   ,  14   
xxxxxx Hello from 53 on r102u35   ,  17   
xxxxxx Hello from 55 on r102u35   ,  19   
xxxxxx Hello from 56 on r102u35   ,  20   
xxxxxx Hello from 58 on r102u35   ,  22   
xxxxxx Hello from 60 on r102u35   ,  24   
xxxxxx Hello from 63 on r102u35   ,  27   
xxxxxx Hello from 64 on r102u35   ,  28   
xxxxxx Hello from 66 on r102u35   ,  30   
xxxxxx Hello from 67 on r102u35   ,  31   
xxxxxx Hell

<hr> 

### Now we are going to build and run a C mpi program

In [14]:
cd /home/tkaiser2/newdriver

/home/tkaiser2/newdriver


In [15]:
ls

bonnkit.ipynb.save                 report.out
c_ex02.c                           report.py*
checkpoint                         save/
checkpoints-1.data-00000-of-00001  slurm-6688638.out
checkpoints-1.data-00000-of-00002  slurm-6688667.out
checkpoints-1.data-00001-of-00002  slurm-6688673.out
checkpoints-1.index                slurm-6688677.out
cupy.ipynb                         slurm-6688703.out
dojup2g                            slurm-6694076.out
dojup2x*                           slurm-6695108.out
dompi.ipynb                        slurm-6696387.out
dpcu.ipynb                         slurm-6705602.out
dpcu.ipynb.save                    slurm-6705733.out
getver*                            slurm-6705788.out
junk/                              slurm-6705811.out
libcublas.so.10@                   slurm-6711763.out
libcudart.so.10.1@                 slurm-6711786.out
libcufft.so.10@                    slurm-6712205.out
libcurand.so.10@                   slurm-6712294.out
libcusolver.so.10@ 

In [16]:
!mpicc c_ex02.c -o c_ex02

In [17]:
srun -n 2 ./c_ex02

' Hello from c process: 0  Numprocs is 2\n Hello from c process: 1  Numprocs is 2\ngetting 1\ni= 200 \n'

In [18]:
out=_
out=out.split("\n")
for o in out:
    print(o)

 Hello from c process: 0  Numprocs is 2
 Hello from c process: 1  Numprocs is 2
getting 1
i= 200 



<hr> 

### We note that we can also start slurm jobs from within a notebook.  The<br> syntax is simply

`
sbatch script
`

### We could, for example, start a notebook on europa and then launch the <br>batch script given above to start a parallel notebook.

